In [1]:
using JuMP
using Gurobi
using StaticArrays
using Rotations
using DataStructures
using GeometryTypes
using DrakeVisualizer
using FileIO
using CoordinateTransformations
using Plots
using PyPlot
using Clp
using ForwardDiff

include("MeshSampling.jl")
using MeshSampling

include("PoseEstimationMathematicalProgram.jl")
using PoseEstimationMathematicalProgram

vis = Visualizer();

In [212]:
# Core parameters we care about

# This many points are sampled on the surface of the model, and
# are used as inliers in the scene cloud.
N_model_points = 8
# This many additional outliers are sampled randomly from a volume around
# the model ([-5, 5]^3) to build the scene cloud.
N_outliers = 2;

Build up information concerning an object of interest:

In [213]:
box = load("cube.obj")

points = decompose(Point{3,Float64}, box)
faces = decompose(Face{3, Int, 0}, box)

# Sample N points from the surface of the mesh
ret = MeshSampling.samplePointsFromMesh(N_model_points, points, faces);
modelPoints = ret[1]
modelPointsFacesGT = ret[2];

Using that model, build up a scene cloud:

In [67]:
T0 = [0.1; 0; 0.5]
R0 = [0.860089 0.174349 -0.479426; 0.244566 0.683829 0.687434; 0.447698 -0.708506 0.545514]

function convertToPointCloudType(points)
    return PointCloud([[points[1, i], points[2, i], points[3, i]] for i=1:size(points, 2)]) 
end

scenePoints = transpose(R0)*modelPoints + repmat(-transpose(R0)*T0, 1, size(modelPoints, 2))

scenePointsOutliers = (rand(3, N_outliers)-0.5)*10.
scenePoints = cat(2, scenePoints, scenePointsOutliers)
scenePointsFacesGT = cat(1, modelPointsFacesGT, zeros(N_outliers, size(modelPointsFacesGT, 2)))
outliersGTAssignment = zeros(N_model_points+N_outliers, 1)
outliersGTAssignment[N_model_points+1:end] = 1


pointcloud = convertToPointCloudType(scenePoints)
box_vis = setgeometry!(vis[:julia][:box_vis], box)
setgeometry!(vis[:julia][:scene_pts], pointcloud);

We want to solve the problem

$$ \min_{R, T, C} \sum_{i \in S} || R s_i + T - C M ||_1 $$

where $C$ selects model vertices $M$ to explain scene point $S_i$, using
only model vertices from a single face of the model at a time, and $R \in SO(3)$.

In [5]:
function buildModelHelper()
    return PoseEstimationMathematicalProgram.constructModel(
        scenePoints, points, faces, use_relaxed_form=true,
    other_solver=GurobiSolver(TimeLimit = 1, FeasRelaxBigM = 20, MIPGap = 0.025, Heuristics = 0.1, Threads = 10, OutputFlag=0),
        closeness=10.0, R0_in=R0, optPhiMax=0.1, optBigNumber=10,
    TimeLimit = 1200, FeasRelaxBigM = 20, MIPGap = 0.025, Heuristics = 0.1, Threads = 5, OutputFlag=0);
end
fullyRelaxedModel = buildModelHelper();

In [209]:
function buildRandomPartialFaceAssignment(scenePoints, faces, points, N)
    nx = size(scenePoints, 2)
    ny = size(faces, 1)
    f_test = zeros(nx, ny) - 1
    for i = 1:min(nx, N)
        f_test[i, rand(1:ny)] = 1 
    end
    return f_test
end;

function computeTransformGivenCorrespondences(scenePoints::Array{Float64}, points, faces,
    f_completed::Array{Float64}, f_outlier_completed::Array{Float64}, R_init::Array{Float64}, T_init::Array{Float64}, C_init::Array{Float64})
    
    function projectRotMat(R::Array{Float64})
        return RotMatrix(Quat(R))
    end
    
    modelVerts = Array{Float64}(3, size(points, 1))
    for i=1:size(points, 1)
        for k=1:3
            modelVerts[k, i] = points[i][k]
        end
    end
    # Build vertex-membership matrix F
    # where F[i, j] == 1 --> 
    #         vertex j is a member of face i
    F = Array{Float64}(size(faces, 1), size(points, 1))
    F .= 0.0
    for i = 1:size(faces, 1)
        for j = faces[i]
            F[i, j] = 1
        end
    end
    
    function computeObjective(args)
        R = reshape(args[1:9], 3, 3)
        T = args[10:12]
        C = reshape(args[13:end], size(scenePoints, 2), size(points, 1))
        total_err = 0.
        for i = 1:size(scenePoints, 2)
            if f_outlier_completed[i] == 0
                total_err += norm(R * scenePoints[:, i] + T - modelVerts * C[i, :], 1)
            end
        end
        return total_err / size(scenePoints, 2)
    end
    
    #=
    m = Model()
    @variable(m, -1. <= R[i=1:3, j=1:3] <= 1., start = R_init[i, j])
    @variable(m, T[i=1:3], start = T_init[i])
    @variable(m, 0. <= C[i=1:size(scenePoints, 2), j=1:size(points, 1)] <= 1., start = C_init[i, j])
    @variable(m, 0. <= obj_helper[1:size(scenePoints, 2)])
    
    for i = 1:size(scenePoints, 2)
        if f_outlier_completed[i] == 0
            @constraint(m, obj_helper[i] == norm(R * scenePoints[:, i] + T - modelVerts * C[i, :]))
            for k = 1:size(faces, 1)
                if f_completed[i, k] == 0
                    @constraint(m, C[i, ])
                    for ind in faces[k]
                        @constraint(m, C[i, ind] == 1.)
                    end
                end
            end
        else
            @constraint(m, obj_helper[i] == 0)
            @constraint(m, C[i, :] == 0)
        end 
    end
    @objective(m, Min, sum( [obj_helper] ))
    =#
    
    R = R_init
    T = T_init
    C = C_init
    args_concat = vcat(reshape(R, 9, 1), T, reshape(C, size(C, 1)*size(C, 2), 1))
    cfg = ForwardDiff.GradientConfig(args_concat)
    
    # Momentum for gradient descent
    velocity = args_concat * 0.0
    err = 0
    C_temp = 0
    for k = 1:100
        err = computeObjective(args_concat) 
        derr = ForwardDiff.gradient(computeObjective, args_concat, cfg)
        #println(k, ": ", err)
        
        grad_component = derr .* derr
        # Compute update
        #println(grad_component)
        delta = velocity - (1./abs(grad_component+1.)) .* derr * 0.05
        args_concat += delta

        # Update velocity with momentum amount
        velocity = delta*0.9
        
        # Reproject R to be a rotation matrix
        r_temp = reshape(args_concat[1:9], 3, 3)
        args_concat[1:9] = reshape(projectRotMat(r_temp), 9, 1)
        # Reproject C to only contain the faces we're allowed to use
        C_temp = reshape(args_concat[13:end], size(C, 1), size(C, 2))
        C_mask = C_temp * 0.
        for i in 1:size(C, 1)
            if f_outlier_completed[i] == 1
                C_mask[i, :] = 0
                break
            else
                for j in 1:size(C, 2)
                    for k in faces[findfirst(f_completed[i, :])]
                        C_mask[i, k] = 1.
                    end
                end
            end
        end
        C_temp = C_temp .* C_mask
        args_concat[13:end] = reshape(C_temp, size(C, 1)*size(C, 2), 1)
    end
    
    R_proj = projectRotMat(R_init)
    
   # println("Final error: ", err)
    
    return (R_proj, T_init)
end

f_test = scenePointsFacesGT
f_outlier_test = outliersGTAssignment
R_test, T_test = computeTransformGivenCorrespondences(scenePoints, points, faces, f_test, f_outlier_test, rand(3, 3), rand(3, 1), rand(size(scenePoints, 2), size(points, 1)));
println(R_test, ", ", T_test)
println(R0, ", ", T0)

[0.361169 -0.159158 0.918818; -0.687168 0.620647 0.377621; -0.630363 -0.767767 0.11479], [0.917409; 0.925262; 0.535491]
[0.860089 0.174349 -0.479426; 0.244566 0.683829 0.687434; 0.447698 -0.708506 0.545514], [0.1,0.0,0.5]


We're going to tackle this by performing branch and bound over the correspondences $f$, where $f[i, k] \implies \text{$s_i$ is on face $k$}$. (You'll have to inspect the detailed problem formulation to find $f$, sorry!)

Given a *partial assignment* of $f$, we can build an LP $LP_{relax}$ by relaxing the unassigned members of $f$ to be continuous. Given its corresponding relaxed solution, including the pseudo-rotation $\hat{R}$, we can:
- *Branch*: Pick a free member of $f$ (i.e. one that is not assigned to 0 or 1 already) and create two new partial assignments, where that assignment is 0, and where that assignment is 1.
- *Provide a lower bound*: The optimal value of the relaxed solution already provides a lower bound. By looking at $R$, $\hat{R}$, and the cost vector of $LP_{relax}$, I suspect we can improve the lower bound by moving backwards (i.e. upwards in cost) along the cost vector until some part of $SO(3)$ is in the null space of the cost vector.
- *Provide an upper bound*: "Complete" the relaxed solution by rounding the correspondences to make all binary assignments truly binary. Solve that to get an intermediate rotation $\hat{R}$. Project $\hat{R}$ into $SO(3)$ to yield $\bar{R}$, and solve $LP_{relax}$ one more time with $R == \bar{R}$ and the correspondences still rounded. (Alternative, maybe more efficient: then account for $R \notin SO(3)$ by projecting $\hat{R} - \bar{R}$ onto the cost vector, where $\bar{R}$ is a valid rotation (chosen by projecting $\hat{R}$ onto $SO(3)$), and adding that to the "completed" solution's cost.)



In [206]:
# Datastructure for the search tree
module CorrespondenceBnB
type PartialAssignmentNode
    # A partial-assignment of f has entries [-1, 0, and 1], 
    # where [0 and 1] are valid assignments, and [-1] is 
    # no-assignment.
    f_assign::Array{Float64} # A partial assignment of f
    f_outlier_assign::Array{Float64} # A partial assignment of f
    
    lower_bound::Float64
    upper_bound::Float64
    
    # List of children nodes (possible empty)
    children::Array{PartialAssignmentNode}
    
    # Parent (or nothing)
    parent::Nullable{PartialAssignmentNode}
    
    # Is this node known to be fathomed?
    is_fathomed::Bool
end
type NodeHistory
    node_hash_table::Array{Array{PartialAssignmentNode}}
end
end

using CorrespondenceBnB

function spawnChildNode(node::CorrespondenceBnB.PartialAssignmentNode)
    return CorrespondenceBnB.PartialAssignmentNode(
    copy(node.f_assign),
    copy(node.f_outlier_assign),
    node.lower_bound,
    node.upper_bound,
    [],
    node,
    false
    )
end
    
    
function printNodeSummary(node::CorrespondenceBnB.PartialAssignmentNode)
    num_pos_assignments = sum( node.f_assign .== 1.0 )
    num_neg_assignments = sum( node.f_assign .== 0.0 )
    num_pos_outlier_assignments = sum( node.f_outlier_assign .== 1.0)
    num_neg_outlier_assignments = sum( node.f_outlier_assign .== 0.0)
    @printf("Node (+%d/-%d | +%d/-%d outlier) [%f <= opt <= %f] ", num_pos_assignments,
    num_neg_assignments, num_pos_outlier_assignments, num_neg_outlier_assignments,
    node.lower_bound, node.upper_bound)
    if (node.is_fathomed)
        print("is fathomed\n")
    else
        print("is not fathomed\n")
    end
end

function processNode(fullyRelaxedModel::PoseEstimationSolveData, node::CorrespondenceBnB.PartialAssignmentNode, best_known_upper_bound::Float64)
    """
    Takes the base model for this search, and a
    node that has been populated with a partial
    assignment and an optional parent.
    
    Computes a lower and upper bound from the partial
    assignment, and generates a list of child nodes
    from all assignments of an unassigned 
    scene point, OR declares this node fathomed.
    """
    
    # Apply the partial assignments
    partiallyAssignedModel = buildModelHelper()
    for (i, val) = enumerate(node.f_assign)
        if (val == 0.0 || val == 1.0)
            @constraint(partiallyAssignedModel.m, partiallyAssignedModel.f[i] == val)
            #println("Assigned ", ind2sub(node.f_assign, i), " to ", val)
        end
    end
    for (i, val) = enumerate(node.f_outlier_assign)
        if (val == 0.0 || val == 1.0)
            @constraint(partiallyAssignedModel.m, partiallyAssignedModel.f_outlier[i] == val)
            #println("Assigned outlier ", ind2sub(node.f_outlier_assign, i), " to ", val)
        end
    end
    status = solve(partiallyAssignedModel.m)
    
    # Lower bound from the relaxation
    node.lower_bound = getobjectivevalue(partiallyAssignedModel.m)
    
    # Check if we're obviated by some other known bound:
    if (best_known_upper_bound != nothing 
        && node.lower_bound >= best_known_upper_bound)
        node.is_fathomed = true
        node.upper_bound = Inf
        node.children = []
        return node
    end
    
    # Upper bound:
    # First, complete the solution to make f binary
    f_completed::Array{Float64} = Array{Float64}(getvalue(partiallyAssignedModel.f))
    f_outlier_completed::Array{Float64} = Array{Float64}(getvalue(partiallyAssignedModel.f_outlier))
    (maxval, index) = findmax(f_completed, 2)
    for (i, k) = enumerate(index)
        if maxval[i] < f_outlier_completed[i]
            f_completed[i, :] = 0.
            f_outlier_completed[i] = 1.
        else
            f_completed[i, :] = 0.
            f_completed[k] = 1.
            f_outlier_completed[i] = 0.
        end
    end
    
    #  Given that completed solution, generate the optimal
    #  transform for each rigid body
    (Rbar, Tbar) = computeTransformGivenCorrespondences(scenePoints, points, faces, f_completed, f_outlier_completed,
    getvalue(partiallyAssignedModel.R), getvalue(partiallyAssignedModel.T), getvalue(partiallyAssignedModel.C))
    
    
    # Build a new model with f forced to take these values
    fCompletedModel = partiallyAssignedModel #buildModelHelper()
    @constraint(fCompletedModel.m, fCompletedModel.f .== f_completed)
    @constraint(fCompletedModel.m, fCompletedModel.f_outlier .== f_outlier_completed)
    @constraint(fCompletedModel.m, fCompletedModel.R .== Rbar)
    status = solve(fCompletedModel.m)
    node.upper_bound = getobjectivevalue(fCompletedModel.m)
    
    # Check if we're fathomed by feasibility by checking bounds
    if (node.upper_bound < node.lower_bound)
        println("Serious problem: upper bound lower than upper bound!")
        node.lower_bound = 0.
        node.upper_bound = 0.
    end
    if (node.upper_bound - node.lower_bound < 0.0001)
        node.is_fathomed = true
        node.children = []
        return node
    end
    
    index_to_assign = 1
    while index_to_assign <= size(node.f_assign, 1)
        if min(node.f_outlier_assign[index_to_assign],
           minimum(node.f_assign[index_to_assign, :], 1)[1]) < 0
            # Not assigned yet!
            break
        else
            index_to_assign += 1
        end
    end
    
    if (index_to_assign > size(node.f_assign, 1))
        # We're fully specified -- this is another way to be fathomed
        node.children = []
        node.is_fathomed = true
        return node
    end
    
    # Finally, populate all children
    
    node.children = []    
    for correspondence_to_assign in 0:size(node.f_assign, 2)
        push!(node.children, spawnChildNode(node))
        if (correspondence_to_assign == 0)
            node.children[correspondence_to_assign+1].f_assign[index_to_assign, :] = 0.
            node.children[correspondence_to_assign+1].f_outlier_assign[index_to_assign] = 1.
        elseif (correspondence_to_assign > 0)
            node.children[correspondence_to_assign+1].f_outlier_assign[index_to_assign] = 0.
            node.children[correspondence_to_assign+1].f_assign[index_to_assign, :] = 0.
            node.children[correspondence_to_assign+1].f_assign[index_to_assign, correspondence_to_assign] = 1.
        end
    end
    
    return node
end

f_test = buildRandomPartialFaceAssignment(scenePoints, faces, points, 0)
f_outlier_test = zeros(size(scenePointsFacesGT, 1), 1)-1
f_test = scenePointsFacesGT
f_outlier_test = outliersGTAssignment
test_node = CorrespondenceBnB.PartialAssignmentNode(f_test, f_outlier_test, 0.0, Inf, [], nothing, false)
printNodeSummary(test_node)
test_node = processNode(fullyRelaxedModel, test_node, 100.);
printNodeSummary(test_node)
println("Children:")
for child in test_node.children
    printNodeSummary(processNode(fullyRelaxedModel, child, 100.))
end

Node (+8/-112 | +2/-8 outlier) [0.000000 <= opt <= Inf] is not fathomed
Final error: 

0.07686860705245555
Node (+8/-112 | +2/-8 outlier) [0.020000 <= opt <= 0.099893] is fathomed
Children:


In [210]:
function compareNodes(a::CorrespondenceBnB.PartialAssignmentNode, b::CorrespondenceBnB.PartialAssignmentNode)
    if a == b
        return true
    end
    
    for i = 1 : size(a.f_assign, 1)
        if a.f_outlier_assign[i] != b.f_outlier_assign
            return false
        end
        
        for j = 1 : size(a.f_assign, 2)
            if a.f_assign[i, j] != b.f_assign[i, j]
                return false
            end
        end
    end
    
    return true
end

# If the given node is in the nodeHistory already, returns false.
# Otherwise, adds it to the node history and returns true.
function tryInsertNodeInTable(nodeHistory::CorrespondenceBnB.NodeHistory, node::CorrespondenceBnB.PartialAssignmentNode)
    # "Hash" by summing the index of the correspondences
    ind = 0
    for i = 1 : size(node.f_assign, 1)
        if node.f_outlier_assign[i] == 1
            continue
        else
            ind += findfirst(node.f_assign[i, :])
        end
    end
    ind = (ind % size(nodeHistory.node_hash_table, 1)) + 1
    
    alreadyInHistory = false
    for oldNode in nodeHistory.node_hash_table[ind]
        if compareNodes(node, oldNode)
            alreadyInHistory = true
            break
        end
    end
    
    if alreadyInHistory
        return false
    else
        push!(nodeHistory.node_hash_table[ind], node)
        return true
    end
end;

In [211]:
module CorrespondenceBnBHistory
type SolveHistory
    elapsed::Array{Float64}
    best_ub::Array{Float64}
    best_lb::Array{Float64}
    searched::Array{Int}
    queued::Array{Int}
    done::Array{Int}
end
end

function updateTree(node::CorrespondenceBnB.PartialAssignmentNode)
    # Updates this node by checking the children's
    # lb and ub, and then recursively calls this on 
    # the parent
    if size(node.children, 1) != 0
        old_lb = node.lower_bound
        node.lower_bound = min(minimum([child.lower_bound for child in node.children]))
        node.upper_bound = max(maximum([child.upper_bound for child in node.children]))
    end
        
    if !isnull(node.parent)
        updateTree(get(node.parent))
    end
end

function runCorrespondenceBranchAndBound(scenePoints, faces, points, f_start, f_outlier_start)
    # Set up root node for our search
    fullyRelaxedModel = buildModelHelper()
    
    # Set up node history
    nodeHistory = CorrespondenceBnB.NodeHistory(repeat([[]], outer=100))
    
    root_node = CorrespondenceBnB.PartialAssignmentNode(f_start, f_outlier_start, 0.0, Inf, [], nothing, false)
    best_known_upper_bound = root_node.upper_bound
    # Remember that Julia's priority queue returns LOWEST priority items first.
    leaf_queue = PriorityQueue(CorrespondenceBnB.PartialAssignmentNode, Float64)
    enqueue!(leaf_queue, root_node, root_node.lower_bound)
    
    # And solve history
    solveHistory = CorrespondenceBnBHistory.SolveHistory([0.0], [root_node.upper_bound], [root_node.lower_bound], [0], [1], [0])
    
    if !tryInsertNodeInTable(nodeHistory, root_node)
        println("Couldn't insert root node in nodeHistory. Bug!")
        return false
    end
    
    incumbent_sol = root_node
    num_fathomed = 0
    for k in 1:10000
        if length(leaf_queue) == 0
            println("Queue empty!")
            @printf("Searched %d, queue %d, fathomed %d, best bounds %f <= x <= %f\n", k, length(leaf_queue), num_fathomed, root_node.lower_bound, best_known_upper_bound)
            print("Incumbent: "); printNodeSummary(incumbent_sol)
            break
        end
        next_node = dequeue!(leaf_queue)
        
        # Process this node to generate its bounds and detect if it's fathomed
        next_node = processNode(fullyRelaxedModel, next_node, best_known_upper_bound)
        
        if (k % 50 == 0)
            print("*******\nLast node : ")
            printNodeSummary(next_node)
            print("Incumbent : ")
            printNodeSummary(incumbent_sol)
            @printf("Searched %d, queue %d, fathomed %d, best bounds %f <= x <= %f\n", k, length(leaf_queue), num_fathomed, root_node.lower_bound, best_known_upper_bound)
        end
        # Did we improve our upper bound estimate?
        best_known_upper_bound = min(next_node.upper_bound, best_known_upper_bound)
        
        updateTree(next_node)
        
        if next_node.is_fathomed
            num_fathomed += 1
            # It may have been fathomed by having a bad lower bound;
            # if it wasn't (it was feasible!), then lb == ub and
            # we should take it as incumbent if it's a better sol.
            # (If lb > best_known_upper_bound, this check will fail, so 
            # we don't need to explicitely handle the first case)
            if next_node.upper_bound <= best_known_upper_bound
                incumbent_sol = next_node
            end
        else
            for child in next_node.children
                if tryInsertNodeInTable(nodeHistory, child)
                    enqueue!(leaf_queue, child, next_node.lower_bound)
                end
            end
        end
    end
end;

f_test = buildRandomPartialFaceAssignment(scenePoints, faces, points, 0)
f_outlier_test = zeros(Float64, size(scenePointsFacesGT, 1), 1) - 1
runCorrespondenceBranchAndBound(scenePoints, faces, points, f_test, f_outlier_test);

*******
Last node : Node (+10/-110 | +0/-10 outlier) [0.000000 <= opt <= 2.318692] is fathomed
Incumbent : Node (+0/-0 | +0/-0 outlier) [0.000000 <= opt <= 2.609267] is not fathomed
Searched 50, queue 419, fathomed 13, best bounds 0.000000 <= x <= 1.834595


*******
Last node : Node (+10/-110 | +0/-10 outlier) [0.000000 <= opt <= 2.299246] is fathomed
Incumbent : Node (+9/-111 | +1/-9 outlier) [0.010000 <= opt <= 1.106140] is fathomed
Searched 100, queue 525, fathomed 51, best bounds 0.000000 <= x <= 1.106140
*******
Last node : Node (+10/-110 | +0/-10 outlier) [0.011471 <= opt <= 2.111371] is fathomed
Incumbent : Node (+9/-111 | +1/-9 outlier) [0.010000 <= opt <= 1.106140] is fathomed
Searched 150, queue 618, fathomed 90, best bounds 0.000000 <= x <= 1.106140
*******
Last node : Node (+7/-77 | +0/-7 outlier) [0.077274 <= opt <= 0.340749] is not fathomed
Incumbent : Node (+9/-111 | +1/-9 outlier) [0.010000 <= opt <= 1.106140] is fathomed
Searched 200, queue 1023, fathomed 105, best bounds 0.000000 <= x <= 1.065934
*******
Last node : Node (+10/-110 | +0/-10 outlier) [0.001579 <= opt <= 2.081988] is fathomed
Incumbent : Node (+9/-111 | +1/-9 outlier) [0.010000 <= opt <= 1.106140] is fathomed
Searched 250, queue 1194, fathomed 138, best boun

In [ ]:
using ProfileView
Profile.clear()  # in case we have any previous profiling data
@profile runCorrespondenceBranchAndBound(scenePoints, faces, points, f_test, f_outlier_test);

In [ ]:
ProfileView.view()